# Before you start :
   - These exercises are related to the Pivot table and correlation lessons.
   - Keep in mind that you need to use some of the functions you learned in the previous lessons.
   - All datasets are provided in the `your-code` folder of this lab.
   - Elaborate your codes and outputs as much as you can.
   - Try your best to answer the questions and complete the tasks and most importantly enjoy the process!!!

#### Import all the libraries that are necessary.

In [1]:
import pandas as pd 
import numpy as np 

# Challenge 1

#### Open the ` Employee.csv` file and name your dataset `employee`.

In [2]:
employee = pd.read_csv('./data/Employee.csv')
employee

,Name,Department,Education,Gender,Title,Years,Salary
0,Jose,IT,Bachelor,M,analyst,1,35
1,Maria,IT,Master,F,analyst,2,30
2,David,HR,Master,M,analyst,2,30
3,Sonia,HR,Bachelor,F,analyst,4,35
4,Samuel,Sales,Master,M,associate,3,55
5,Eva,Sales,Bachelor,F,associate,2,55
6,Carlos,IT,Master,M,VP,8,70
7,Pedro,IT,Phd,M,associate,7,60
8,Ana,HR,Master,F,VP,8,70


#### What's the mean salary by department?
Hint : There are 2 ways to do it - one way is with and the other way is without pivot tables. Do it both ways. 


In [3]:
# Method 1
employee.pivot_table(index="Department", values='Salary', aggfunc=np.mean).rename(columns = {'Salary':'Avg Salary'})

,Avg Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


In [4]:
# Method 2
# your answer here
employee.groupby(by = 'Department').agg({'Salary':'mean'}).rename(columns = {'Salary':'Avg Salary'})

,Avg Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


#### We want now to group the data by more than one column. Use Pandas pivot table function to select the mean salary by department and title.

In [5]:
employee.pivot_table(index=["Department", "Title"], values='Salary', aggfunc=np.mean).rename(columns = {'Salary':'Avg Salary'})

Avg Salary
Department Title                
HR         VP               70.0
           analyst          32.5
IT         VP               70.0
           analyst          32.5
           associate        60.0
Sales      associate        55.0

#### We want to display the number of years of work experience and the salary for each Title. Add Years as a column in your pivot table.
*Hint: Use Pandas doc(see References in README.md) to explore the pivot_table() arguments*

In [6]:
employee.pivot_table(index=["Department", "Title"], values='Salary', columns = 'Years', fill_value = '-')

Years                  1   2   3   4   7   8
Department Title                            
HR         VP          -   -   -   -   -  70
           analyst     -  30   -  35   -   -
IT         VP          -   -   -   -   -  70
           analyst    35  30   -   -   -   -
           associate   -   -   -   -  60   -
Sales      associate   -  55  55   -   -   -

#### We want to know the information about the mean salary and number of employees in each department and for each job title.

In [7]:
employee.pivot_table(index =["Department"],  values=["Salary", 'Name'], columns=["Title"], 
                     aggfunc= {'Salary':'mean', 'Name': 'count' }, fill_value='-').rename(
    columns={'Name': 'Nb of Employees', 'Salary': 'Avg. Salary'})

Nb of Employees                   Avg. Salary                  
Title                   VP analyst associate          VP analyst associate
Department                                                                
HR                       1       2         -          70    32.5         -
IT                       1       2         1          70    32.5        60
Sales                    -       -         2           -       -        55

#### We want to know for each department and for each job title the Salary's median, minimum, maximum and standard deviation. Comment your results.

In [16]:
employee_stats = employee.pivot_table(index = ["Department"], columns = ["Title"], 
                                      values = ['Salary', 'Name'], aggfunc = (np.median, np.min, np.max, np.std))
employee_stats

Salary                                                           \
             amax                    amin                   median           
Title          VP analyst associate    VP analyst associate     VP analyst   
Department                                                                   
HR           70.0    35.0       NaN  70.0    30.0       NaN   70.0    32.5   
IT           70.0    35.0      60.0  70.0    30.0      60.0   70.0    32.5   
Sales         NaN     NaN      55.0   NaN     NaN      55.0    NaN     NaN   

                                          
                           std            
Title      associate   analyst associate  
Department                                
HR               NaN  3.535534       NaN  
IT              60.0  3.535534       NaN  
Sales           55.0       NaN       0.0

#### Based on your comments, fill the missing values with an appropriate value.

In [10]:
# From the table, we can see that:
# The VPs from each department have the same salary.
# The analyst for each department earn an average of 32.5 
# The associates earns more in IT than Sales

employee_stats.fillna(employee_stats.mean())

Salary                                                           \
             amax                    amin                   median           
Title          VP analyst associate    VP analyst associate     VP analyst   
Department                                                                   
HR           70.0    35.0      57.5  70.0    30.0      57.5   70.0    32.5   
IT           70.0    35.0      60.0  70.0    30.0      60.0   70.0    32.5   
Sales        70.0    35.0      55.0  70.0    30.0      55.0   70.0    32.5   

                                          
                           std            
Title      associate   analyst associate  
Department                                
HR              57.5  3.535534       0.0  
IT              60.0  3.535534       0.0  
Sales           55.0  3.535534       0.0

#### The stake holders want to know for each department the number of employees and how much money is spend on salaries. Could you provide that information? 

In [15]:
employee.pivot_table(index = ["Department"], values = ['Name', "Salary"], 
                     aggfunc = {'Name': 'count', 'Salary': 'sum'}).rename(
    columns = {'Name': 'Nb of Employees', 'Salary': 'Total Salary'})

,Nb of Employees,Total Salary
Department,,
HR,3,135
IT,4,195
Sales,2,110


#### For each Department and Title, we want to know the the total years of work experience and the mean salary.

In [21]:
employee.pivot_table(index = ['Department'], columns = ['Title'], values = ['Years', 'Salary'],
                    aggfunc = {'Years': 'count', 'Salary': 'mean'}, fill_value = '-').rename(
    columns = {'Years': 'Total Years', 'Salary': 'Avg. Salary'})

Avg. Salary                   Total Years                  
Title               VP analyst associate          VP analyst associate
Department                                                            
HR                  70    32.5         -           1       2         -
IT                  70    32.5        60           1       2         1
Sales                -       -        55           -       -         2

### Bonus

#### We now want to compute the mean salary after removing the maxima for each Department. Create the appropriate pivot table.

Hint: Write a custom function.

# Challenge 2

#### Open the ` Fitbit.csv` file and name your dataset `fitbit`.

In [ ]:
fitbit = pd.read_csv('./data/Fitbit.csv')
fitbit

#### Explore you dataset in terms of data types and descriptive statistics.
Hint: Use Pandas functions from previous lectures.

In [ ]:
fitbit.dtypes

In [ ]:
fitbit.describe()

#### You suspect that there must be a linear relationship between the Minutes Very Active and the Steps. Compute the correlation between these variables.

In [ ]:
fitbit['Steps'].corr(fitbit['Minutes Very Active'])

#### Use matplotlib to visually plot Minutes Very active vs Steps. 

##### Hint : import matplotlib.pyplot as plt (See previous lessons)#####
- Use plt.scatter(x=df['col_name_1'], y=df['col_name_2'])

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x = fitbit['Steps'], y = fitbit['Minutes Very Active'])
plt.show()

#### What can you say about Minute Very Active and Steps? Write a comment below

In [ ]:
# There is no strong relation between the Minute Very Active and Steps - the correlation coefficient is only 0.57.
# This is confirmed by the plot: the data is quite disperse

#### We also suspect that there must be a linear relationship between the Minutes Sedentary and the Steps. Compute the correlation between these variables.

In [ ]:
fitbit['Steps'].corr(fitbit['Minutes Sedentary'])

#### Use matplotlib to visually plot Minutes Sedentary vs Steps. Based on the results of the computed correlation and the plot. What can you say about these 2 variables?

In [ ]:
plt.scatter(x = fitbit['Steps'], y = fitbit['Minutes Sedentary'])

# There is not correlation between Minutes Sedentary vs Steps which make sense because sedentary means that you are not active

#### We also suspect that there must be a linear relationship between the MinutesOfSleep and the Steps. Compute the correlation between these variables.

In [ ]:
fitbit['Steps'].corr(fitbit['MinutesOfSleep'])

#### Use matplotlib to visually plot MinutesOfSleep vs Steps. Based on the results of the computed correlation and the plot. What can you say about these 2 variables?

In [ ]:
plt.scatter(x = fitbit['Steps'], y = fitbit['MinutesOfSleep'])
# There is not correlation between Minutes of Sleep vs Steps which make sense because you are not active when sleeping

#### Select a subset of your dataset with the columns below and compute the correlation matrix. Save the results in the variable `cor_fit`
column= ['Calorie burned', 'Steps','Floors','Minutes Sedentary','Minutes Very Active', 'MinutesOfSleep']

In [ ]:
sub_fitbit = fitbit[['Calorie burned', 'Steps','Floors','Minutes Sedentary','Minutes Very Active', 'MinutesOfSleep']]
cor_fit = sub_fitbit.corr()
cor_fit

#### Based on the result of the correlation matrix. What are the highly correlated features with the variable Steps?
*Hint: From the results above select the variable Steps and order your results in descending order (use Pandas sort_values function)*

In [ ]:
cor_fit.loc['Steps'].sort_values(ascending = False)
# The highest correlation is between steps and Minutes Very Active but it is still not really strong

# Challenge 3

#### Open the `Time_Grades.csv` file and name your dataset `time_grades`.

In [ ]:
time_grades = pd.read_csv('./data/Time_Grades.csv')
time_grades

#### Show visually the frequency distribution of the time_grades dataset.

In [ ]:
time_grades.plot.bar()

#### Is there a correlation between study time and Grade? Use both Pearson and Spearman correlation and comment your results.

In [ ]:
pers_corr = time_grades['Study time'].corr(time_grades['Grade'], method = 'pearson')
spear_corr = time_grades['Study time'].corr(time_grades['Grade'], method = 'spearman')

print(pers_corr, spear_corr)
# From the Spearman method we can see that the data is moving in the same direction
# From the Pearson method we can see that there is a linear relashionship between study time and grades

#### Use matplotlib to visually plot Study time vs Grade. Based on the results of the computed correlation and the plot. What can you say about these 2 variables?

In [ ]:
plt.scatter(x = 'Grade', y = 'Study time', data = time_grades)

#### Could you explain the difference between Pearson and Spearman correlation? (see References in README.md) 

In [ ]:
# Spearman measures relationships between variables that are not necessarily linear: it's looking at whether 
# the data is moving in the same or the opposite direction.
# Pearson method measures linear relationships between variables.